# momondo.Fr




In [ ]:
###https://www.momondo.fr/flight-search/PAR-FRA/2023-11-22/2023-11-28?sort=price_a

In [3]:
# Scrappe and Clean
!pip install selenium
!pip install pandas
!pip install urllib3
!pip install requests
!pip install fake-useragent
!pip install numpy


## Import

In [2]:


# Scrappe and Clean
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time

from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta


import pandas as pd 
import numpy as np 


from selenium.webdriver.common.action_chains import ActionChains


# Scrap

In [7]:
ua = UserAgent()
user_agent = ua.random

villes = ["BER", "PAR", "ROM", "MAD", "LON", "LIS", "AMS", "BRU", "DUB", "CPH", "OSL", "STO", "HEL", "REK", "MOW", "WAW", "PRG", "BUD", "VIE", "ATH", "BUH", "SOF", "BEG", "ZAG", "TIA", "SKP", "TGD", "PRN", "TLL", "RIX", "VNO", "KIV", "MSQ", "KBP", "KBP", "TBS", "EVN", "BAK", "ANK", "ECN", "MLA", "VAT", "MCM", "RMI"]
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

all_data = []

logging.basicConfig(filename='scraping.log', level=logging.ERROR)

# Date initiale
start_date = datetime(2023, 11, 22)
end_date = datetime(2023, 11, 28)

# Date finale (2 mois plus tard)
end_date_final = datetime(2024, 1, 22)

# Boucle sur les semaines
while start_date <= end_date_final:
    for i in range(0, len(villes), 2):
        ville1 = villes[i]
        ville2 = villes[i + 1]

        print(f'Début scrape des villes {ville1} et {ville2} pour la semaine du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")}')

        
        driver = webdriver.Firefox(options=options)
        try:
            driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
        except NewConnectionError:
            logging.error(f'Could not establish connection for cities {ville1} and {ville2}, retrying...')
            time.sleep(10)
            try:
                driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
            except Exception as e:
                logging.error(f'Failed to scrape cities {ville1} and {ville2}: {e}')
                driver.quit()
                continue
        time.sleep(2)

        # Fonction auxiliaire pour récupérer des données
        def extract_data(element, xpath):
            try:
                data = element.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data
        time.sleep(2)
        # Récupérer le contenu des divs JWEO et xdW8 dans c3J0r-container
        # Récupérer le contenu des éléments
        voyages = driver.find_elements(By.XPATH, './/div[contains(@class, "nrc6-inner")]')


        for voyage in voyages:
        # Récupérer le contenu des éléments pour chaque voyage
            horaires = extract_data(voyage, './/div[@class="vmXl vmXl-mod-variant-large"]')
            trajet = extract_data(voyage, './/div[@class="c_cgF c_cgF-mod-variant-full-airport"]')
            type_trajet = extract_data(voyage, './/span[@class="JWEO-stops-text"]')
            temps_trajet = extract_data(voyage, './/div[@class="xdW8 xdW8-mod-full-airport"]/div[@class="vmXl vmXl-mod-variant-default"]')
            prix = extract_data(voyage, './/div[@class="f8F1-price-text"]')
            type_voyage = extract_data(voyage, './/div[@class="aC3z-name"]')
            Compagnie = extract_data(voyage, './/div[@class="J0g6-operator-text"]')
        # Imprimer les résultats pour chaque voyage
            print("Horaires:", horaires)
            print("Trajet:", trajet)
            print("Type de trajet:", type_trajet)
            print("Temps de trajet:", temps_trajet)
            print("Prix:", prix)
            print("Type de voyage:", type_voyage)
            print("Compagnie de vol:", Compagnie)
            print("========================================")
        # Fermeture du webdriver
        driver.quit()

    # Passer à la semaine suivante
    start_date += timedelta(weeks=1)
    end_date += timedelta(weeks=1)

print('Fin du scraping')

Début scrape des villes BER et PAR pour la semaine du 2023-11-22 au 2023-11-28
Horaires: 20:15 – 22:05
Trajet: BERAéroport de Berlin Brandebourg-CDGAéroport Charles-de-Gaulle
Type de trajet: direct
Temps de trajet: 1h 50min
Prix: 118 €
Type de voyage: Light
Compagnie de vol: Air France
Horaires: 09:05 – 10:55
Trajet: BERAéroport de Berlin Brandebourg-CDGAéroport Charles-de-Gaulle
Type de trajet: direct
Temps de trajet: 1h 50min
Prix: 86 €
Type de voyage: Standard
Compagnie de vol: easyJet
Horaires: 10:10 – 21:45
Trajet: BERAéroport de Berlin Brandebourg-BVAAéroport de Beauvais-Tillé
Type de trajet: direct
Temps de trajet: 11h 35min
Prix: 92 €
Type de voyage: Value, Standard
Compagnie de vol: Ryanair, easyJet
Horaires: 09:05 – 10:55
Trajet: BERAéroport de Berlin Brandebourg-CDGAéroport Charles-de-Gaulle
Type de trajet: direct
Temps de trajet: 1h 50min
Prix: 96 €
Type de voyage: Standard
Compagnie de vol: easyJet
Horaires: 09:05 – 10:55
Trajet: BERAéroport de Berlin Brandebourg-CDGAéropo

KeyboardInterrupt: 

## Tache à réaliser : Clean Data

In [ ]:
import time
import logging
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
import csv

ua = UserAgent()
user_agent = ua.random

villes = ["BER", "PAR", "ROM", "MAD", "LON", "LIS", "AMS", "BRU", "DUB", "CPH", "OSL", "STO", "HEL", "REK", "MOW", "WAW", "PRG", "BUD", "VIE", "ATH", "BUH", "SOF", "BEG", "ZAG", "TIA", "SKP", "TGD", "PRN", "TLL", "RIX", "VNO", "KIV", "MSQ", "KBP", "KBP", "TBS", "EVN", "BAK", "ANK", "ECN", "MLA", "VAT", "MCM", "RMI"]
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

all_data = []

logging.basicConfig(filename='scraping.log', level=logging.ERROR)


with open('scraping_results.csv', 'w', newline='') as csv_file:
    
    csv_writer = csv.writer(csv_file)
    
    csv_writer.writerow(["VILLE_DEPART", "VILLE_ARRIVEE", "Horaires", "Trajet", "Type de trajet", "Temps de trajet", "Prix", "Compagnie de vol"])


start_date = datetime(2023, 11, 22)
end_date = datetime(2023, 11, 28)


end_date_final = datetime(2024, 1, 22)


while start_date <= end_date_final:
    for i in range(0, len(villes), 2):
        ville1 = villes[i]
        ville2 = villes[i + 1]

        print(f'Début scrape des villes {ville1} et {ville2} pour la semaine du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")}')

        driver = webdriver.Firefox(options=options)
        try:
            driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
        except NewConnectionError:
            logging.error(f'Could not establish connection for cities {ville1} and {ville2}, retrying...')
            time.sleep(10)
            try:
                driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
            except Exception as e:
                logging.error(f'Failed to scrape cities {ville1} and {ville2}: {e}')
                driver.quit()
                continue
        time.sleep(2)

        
        def extract_data(element, xpath):
            try:
                data = element.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data

       
        voyages = driver.find_elements(By.XPATH, './/div[contains(@class, "nrc6-inner")]')

        for voyage in voyages:
            
            horaires = extract_data(voyage, './/div[@class="vmXl vmXl-mod-variant-large"]')
            trajet = extract_data(voyage, './/div[@class="c_cgF c_cgF-mod-variant-full-airport"]')
            type_trajet = extract_data(voyage, './/span[@class="JWEO-stops-text"]')
            temps_trajet = extract_data(voyage, './/div[@class="xdW8 xdW8-mod-full-airport"]/div[@class="vmXl vmXl-mod-variant-default"]')
            prix = extract_data(voyage, './/div[@class="f8F1-price-text"]')
            type_voyage = extract_data(voyage, './/div[@class="aC3z-name"]')
            Compagnie = extract_data(voyage, './/div[@class="J0g6-operator-text"]')

            csv_writer.writerow([ville1, ville2, horaires, trajet, type_trajet, temps_trajet, prix, Compagnie])

            
            print("Horaires:", horaires)
            print("Trajet:", trajet)
            print("Type de trajet:", type_trajet)
            print("Temps de trajet:", temps_trajet)
            print("Prix:", prix)
            print("Type de voyage:", type_voyage)
            print("Compagnie de vol:", Compagnie)
            print("========================================")

       
        driver.quit()

    
    start_date += timedelta(weeks=1)
    end_date += timedelta(weeks=1)

print('Fin du scraping')


In [7]:
from itertools import combinations
ua = UserAgent()
user_agent = ua.random

villes = ["BER", "PAR", "ROM", "MAD", "LON", "LIS", "AMS", "BRU", "DUB", "CPH", "OSL", "STO", "HEL", "REK", "MOW", "WAW", "PRG", "BUD", "VIE", "ATH", "BUH", "SOF", "BEG", "ZAG", "TIA", "SKP", "TGD", "PRN", "TLL", "RIX", "VNO", "KIV", "MSQ", "KBP", "KBP", "TBS", "EVN", "BAK", "ANK", "ECN", "MLA", "VAT", "MCM", "RMI"]
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

all_data = []

logging.basicConfig(filename='scraping.log', level=logging.ERROR)

# Date initiale
start_date = datetime(2023, 11, 22)
end_date = datetime(2023, 11, 28)

# Date finale (2 mois plus tard)
end_date_final = datetime(2024, 1, 22)
city_pairs = list(combinations(villes, 2))
# Boucle sur les semaines
while start_date <= end_date_final:
    for pair in city_pairs:
        ville1, ville2 = pair

        print(f'Début scrape des villes {ville1} et {ville2} pour la semaine du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")}')

        
        driver = webdriver.Firefox(options=options)
        try:
            driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
        except NewConnectionError:
            logging.error(f'Could not establish connection for cities {ville1} and {ville2}, retrying...')
            time.sleep(10)
            try:
                driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
            except Exception as e:
                logging.error(f'Failed to scrape cities {ville1} and {ville2}: {e}')
                driver.quit()
                continue
        time.sleep(5)

        # Fonction auxiliaire pour récupérer des données
        def extract_data(element, xpath):
            try:
                data = element.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data
        
        # Récupérer le contenu des divs JWEO et xdW8 dans c3J0r-container
        # Récupérer le contenu des éléments
        voyages = driver.find_elements(By.XPATH, './/div[contains(@class, "nrc6-inner")]')
        time.sleep(5)

        for voyage in voyages:
            time.sleep(3)
        # Récupérer le contenu des éléments pour chaque voyage
            horaires = extract_data(voyage, './/div[@class="vmXl vmXl-mod-variant-large"]')
            trajet = extract_data(voyage, './/div[@class="c_cgF c_cgF-mod-variant-full-airport"]')
            type_trajet = extract_data(voyage, './/span[@class="JWEO-stops-text"]')
            temps_trajet = extract_data(voyage, './/div[@class="xdW8 xdW8-mod-full-airport"]/div[@class="vmXl vmXl-mod-variant-default"]')
            prix = extract_data(voyage, './/div[@class="f8F1-price-text"]')
            type_voyage = extract_data(voyage, './/div[@class="aC3z-name"]')
            Compagnie = extract_data(voyage, './/div[@class="J0g6-operator-text"]')
        # Imprimer les résultats pour chaque voyage
            print("Horaires:", horaires)
            print("Trajet:", trajet)
            print("Type de trajet:", type_trajet)
            print("Temps de trajet:", temps_trajet)
            print("Prix:", prix)
            print("Type de voyage:", type_voyage)
            print("Compagnie de vol:", Compagnie)
            print("========================================")
            
            data_entry = {"Date_Départ":start_date.strftime("%Y-%m-%d"),"Date_arrivée":end_date.strftime("%Y-%m-%d"),  "VILLE_DEPART": ville1,"VILLE_ARRIVEE": ville2,"Horaires": horaires,"Trajet": trajet,"Type de trajet": type_trajet,"Temps de trajet": temps_trajet,"Prix": prix,"Compagnie de vol": Compagnie}
            
            all_data.append(data_entry)
            print("done")
            
        # Fermeture du webdriver
        driver.quit()

    # Passer à la semaine suivante
    start_date += timedelta(weeks=1)
    end_date += timedelta(weeks=1)
df = pd.DataFrame(all_data)
                          # Specify the CSV file path
csv_file = "data_momo.csv"

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
print('Fin du scraping')

Début scrape des villes BER et PAR pour la semaine du 2023-11-22 au 2023-11-28
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 
done
Horaires: 
Trajet: 
Type de trajet: 
Temps de trajet: 
Prix: 
Type de voyage: 
Compagnie de vol: 

KeyboardInterrupt: 

In [ ]:
df.head()